In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/strokepredictionrf/scikitlearn/rf/2/random_forest_model.pkl


In [2]:
!pip install -U keras-nlp
!pip install -U keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.4/508.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
  Attempting uninstall: keras-nlp
    Found existing installation: keras-nlp 0.8.2
    Uninstalling keras-nlp-0.8.2:
      Successfully uninstalled keras-nlp-0.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Success

In [3]:
import keras
import keras_nlp
import numpy as np

2024-04-15 16:47:26.122567: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 16:47:26.122665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 16:47:26.227287: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import os

# The Keras 3 distribution API is only implemented for the JAX backend for now
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate 100% of TPU memory to minimize memory fragmentation
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

In [5]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [6]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

In [7]:
gemma_lm.compile()

In [8]:
import pickle 
with open("/kaggle/input/strokepredictionrf/scikitlearn/rf/2/random_forest_model.pkl", 'rb') as file:
    clf_re = pickle.load(file)

In [9]:
import numpy as np

# Mapping dictionaries
gender_mapping = {'Male': 1, 'Female': 0}
hypertension_mapping = {'No': 0, 'Yes': 1}
heart_disease_mapping = {'No': 0, 'Yes': 1}
ever_married_mapping = {'No': 0, 'Yes': 1}
work_type_mapping = {'Private': 0, 'Self-employed': 1, 'Govt_job': 2, 'children': 3}
residence_type_mapping = {'Urban': 0, 'Rural': 1}
smoking_status_mapping = {'Non-smoker': 0, 'Smoker': 1, 'Ex-smoker': 2, 'Unknown': 3}


In [10]:
def get_user_input():
    # Gender
    gender = input("Enter Gender (Male/Female): ")
    gender_code = gender_mapping[gender]
    
    # Age
    age = int(input("Enter Age: "))
    
    # Hypertension
    hypertension = input("Enter Hypertension (No/Yes): ")
    hypertension_code = hypertension_mapping[hypertension]
    
    # Average Glucose Level
    avg_glucose_level = float(input("Enter Average Glucose Level: "))
    
    # BMI
    bmi = float(input("Enter BMI: "))
    
    # Residence Type
    Residence_type = input("Enter Residence Type (Urban/Rural): ")
    Residence_type_code = residence_type_mapping[Residence_type]
    
    # Smoking Status
    smoking_status = input("Enter Smoking Status (Non-smoker/Smoker/Ex-smoker/Unknown): ")
    smoking_status_code = smoking_status_mapping[smoking_status]
    
    # Heart Disease
    heart_disease = input("Enter Heart Disease (No/Yes): ")
    heart_disease_code = heart_disease_mapping[heart_disease]
    
    # Ever Married
    ever_married = input("Enter Ever Married (No/Yes): ")
    ever_married_code = ever_married_mapping[ever_married]
    
    # Work Type
    work_type = input("Enter Work Type (Private/Self-employed/Govt_job/children): ")
    work_type_code = work_type_mapping[work_type]
    
    # Prepare data
    data = np.array([[gender_code, age, hypertension_code, heart_disease_code, ever_married_code, work_type_code, Residence_type_code, avg_glucose_level, bmi, smoking_status_code]])
    
    return data

# Call the function to get user input and prepare data
data = get_user_input()

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
y_pred = clf_re.predict(data)

In [ ]:
def generate_text_Gemma(shap_values, feature_names):

    # Extract feature names from the DataFrame
    first_values = [pair[0] for pair in shap_values[0]]
    positive_indices = [i for i, value in enumerate(first_values) if value > 0]
    prompt = ""
    for i in positive_indices:
        i = int(i)
        feature_shap_values = shap_values[0][i]
        mean_shap_value = np.mean(feature_shap_values)
        if mean_shap_value > 0:
            prompt += f"- {feature_names[i]}"
        else:
            prompt += f"- {feature_names[i]}"

    prompt += "As a doctor, given the following information about a patient, can you provide a explanation of how these features contribute to stroke risk?\n"

  
    prompt = template.format(
    instruction=prompt,
    response="",
    )

    generated_text = gemma_lm.generate(prompt, max_length=512)

    return generated_text

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
explainer = shap.Explainer(clf_re)
shap_values = explainer.shap_values(data)
    
# Generate text explanation using the GPT-2 model
feature_names = ['Gender', 'Age', 'Hypertension', 'Heart Disease', 'Ever Married', 'Work Type', 'Residence Type', 'Average Glucose Level', 'BMI', 'Smoking Status']
explanation_text = generate_text_Gemma(shap_values, feature_names)
    


In [ ]:
# Display the prediction, SHAP values, and text explanation
print(f"Prediction: {y_pred[0]}")
print(f"Explanation: {explanation_text}")